# Crowd Counting Notebook
## Directions: 
1. Click on `Runtime` -> `Run All`. 
2. Wait for Cell #1 to complete processing. The last line of the output should be `/content/CrowdCounting`
3. Scroll down to Cell #2 and click on the `Choose Files` button.
4. Choose the file you want to upload and click open/upload.
5. Wait for the code to run and you should have the count printed out at the bottom.
6. If you want to test a different image, you just have to run cell #2. To do so, click the play button (triangle in circle) on the left of cell #2. You might have to scroll down to the cell to make the play button appear. Then follow steps 4-5. 

In [ ]:
### CELL #1

!gdown "1v9u-1ll44e1O9R2dKCOPDykM6nPRVvpB"
!gdown "1v9ym764SlUe5Cn8j2scR726eCOvQ1mP0"
!git clone https://github.com/DSSR2/CrowdCounting.git

!mv best_M-SegNet.pth CrowdCounting/weights
!mv best_M-SFANet.pth CrowdCounting/weights

%cd CrowdCounting
!mkdir images

import sys
sys.path.append("./CrowdCounting/")
sys.path.append("./CrowdCounting/datasets/")
sys.path.append("./CrowdCounting/models/")

import cv2
from PIL import Image
import numpy as np
import warnings
warnings.filterwarnings("ignore")

import torch
from torchvision import transforms
from tqdm import tqdm

from datasets.crowd import Crowd
from models import M_SFANet_UCF_QNRF
from models import M_SegNet_UCF_QNRF
from viz import vis_densitymap

In [ ]:
### CELL #2

from google.colab import files
uploaded = files.upload()
file_names = list(uploaded.keys())

trans = transforms.Compose([transforms.ToTensor(), 
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                           ])

img = Image.open(file_names[0]).convert('RGB')
height, width = img.size[1], img.size[0]
height = round(height / 16) * 16
width = round(width / 16) * 16
img = cv2.resize(np.array(img), (width,height), cv2.INTER_CUBIC)
img = trans(Image.fromarray(img))[None, :]

m1 = M_SFANet_UCF_QNRF.Model()
m2 = M_SegNet_UCF_QNRF.Model()

models = [m1, m2]

for i in models:
    i.cuda()

m1.load_state_dict(torch.load("./weights/best_M-SFANet.pth"))
m2.load_state_dict(torch.load("./weights/best_M-SegNet.pth"))


den_maps = []
sum_vals = 0
for idx, i in enumerate(models):
    i.eval()
    with torch.no_grad():
        outputs = i(img.cuda())
        sum_vals += torch.sum(outputs).item() 
        den_maps.append(outputs)
    
print("\n\nCrowd Count: ", np.round(sum_vals/2))


vis_densitymap(img.cpu(), den_maps[0].cpu().detach().numpy(), int(torch.sum(den_maps[0]).item()), file_names[0])